In [1]:
install.packages("tidyverse")
install.packages("caTools")
install.packages("pROC")
install.packages("ROCR")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependency ‘stringr’

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [2]:
library(tidyverse)
library(caTools)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.1     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.4.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
# importing the Framingham dataset
framingham = read.csv("./files/framingham.csv")
str(framingham)

'data.frame':	4240 obs. of  16 variables:
 $ male           : int  1 0 1 0 0 0 0 0 1 1 ...
 $ age            : int  39 46 48 61 46 43 63 45 52 43 ...
 $ education      : int  4 2 1 3 3 2 1 2 1 1 ...
 $ currentSmoker  : int  0 0 1 1 1 0 0 1 0 1 ...
 $ cigsPerDay     : int  0 0 20 30 23 0 0 20 0 30 ...
 $ BPMeds         : int  0 0 0 0 0 0 0 0 0 0 ...
 $ prevalentStroke: int  0 0 0 0 0 0 0 0 0 0 ...
 $ prevalentHyp   : int  0 0 0 1 0 1 0 0 1 1 ...
 $ diabetes       : int  0 0 0 0 0 0 0 0 0 0 ...
 $ totChol        : int  195 250 245 225 285 228 205 313 260 225 ...
 $ sysBP          : num  106 121 128 150 130 ...
 $ diaBP          : num  70 81 80 95 84 110 71 71 89 107 ...
 $ BMI            : num  27 28.7 25.3 28.6 23.1 ...
 $ heartRate      : int  80 95 75 65 85 77 60 79 76 93 ...
 $ glucose        : int  77 76 70 103 85 99 85 78 79 88 ...
 $ TenYearCHD     : int  0 0 0 1 0 0 1 0 0 0 ...


In [4]:
# Split the dataset into a training, and testing datasets
set.seed(1000)
split = sample.split(framingham$TenYearCHD, 0.65)
train = subset(framingham, split)
test = subset(framingham, !split)

In [5]:
# Build a logistic regression model to predict the 10-year-CHD
framinghamLog = glm(TenYearCHD ~ ., data=train, family="binomial")
summary(framinghamLog)


Call:
glm(formula = TenYearCHD ~ ., family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8487  -0.6007  -0.4257  -0.2842   2.8369  

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -7.886574   0.890729  -8.854  < 2e-16 ***
male             0.528457   0.135443   3.902 9.55e-05 ***
age              0.062055   0.008343   7.438 1.02e-13 ***
education       -0.058923   0.062430  -0.944  0.34525    
currentSmoker    0.093240   0.194008   0.481  0.63080    
cigsPerDay       0.015008   0.007826   1.918  0.05514 .  
BPMeds           0.311221   0.287408   1.083  0.27887    
prevalentStroke  1.165794   0.571215   2.041  0.04126 *  
prevalentHyp     0.315818   0.171765   1.839  0.06596 .  
diabetes        -0.421494   0.407990  -1.033  0.30156    
totChol          0.003835   0.001377   2.786  0.00533 ** 
sysBP            0.011344   0.004566   2.485  0.01297 *  
diaBP           -0.004740   0.008001  -0.592 

In [6]:
# Checking the accuracy of the model on the testing data set, and building a confusion matrix
predictTest = predict(framinghamLog, type="response", newdata=test)
summary(predictTest)
tbl = table(test$TenYearCHD, predictTest > 0.5)
print(tbl)
# Baseline accuracy is always predicting no heart disease at 10 years
baselineacc = (tbl[1]+tbl[3]) / (tbl[1]+tbl[2]+tbl[3]+tbl[4])
# Model accuracy
acc = (tbl[1]+tbl[4]) / (tbl[1]+tbl[2]+tbl[3]+tbl[4])

print(baselineacc)
print(acc)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
0.01641 0.06603 0.11430 0.15006 0.20081 0.72958     211 

   
    FALSE TRUE
  0  1069    6
  1   187   11
[1] 0.8444619
[1] 0.8483896


In [9]:
# One library for ROC AUC curve
library(pROC)
#citation("pROC")
roc = roc(test$TenYearCHD, predictTest)
auc(roc)
ci(roc)
plot(roc)

Area under the curve: 0.7421

95% CI: 0.7052-0.779 (DeLong)

In [23]:
# Another library
library(ROCR)
ROCRpred = prediction(predictTest, test$TenYearCHD)
ROCRperf1 = performance(ROCRpred, "auc")
print(ROCRperf1@y.values)

ROCRperf2 = performance(ROCRpred, "tpr", "fpr")
plot(ROCRperf2)

[[1]]
[1] 0.7421095

